# Introduction to `pyspark.sql.DataFrame`s

Adapted from [Databrick's tutorial](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html)

In [56]:
# import pyspark class Row from module sql
from pyspark.sql import *

## What is spark?

* Build for the Hadoop platform
* Replacement of MapReduce
* Second-generation optimization
    * Lazy
    * Optimized
    * Persistent data structures
* Written in scala

## Ok ... so what's Hadoop?

* Distributed computing platform
* [Used by lots of companies](https://wiki.apache.org/hadoop/PoweredBy)
* Becoming an inductry standard


## What is `pyspark`?

* Python interface to spark
* Needs a spark session
    * `session` $\leftrightarrow$ spark


## Step 0 - Create a spark session

* `pyspark` communicates with `spark` through a session
* Similar to `sqlalchemy` session.

In [2]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

## Overview -  `pyspark.DataFrame`

* A `DataFrame` is a collection of `Row`s
* `Row`s can be distributed over many machines
* `spark`
    * Hides the messy details
    * Optimizes operations

## Creating a `Row` of data

* Use the `Row` class
* Pass data using keywords
    * key == column name
    * value == cell value

In [3]:
department1 = Row(id='123456', name='Computer Science')
department1

Row(id='123456', name='Computer Science')

## Unpacking a `Row` dictionary

* Data is in a row dictionary
* Unpack keywords using `**`

In [4]:
dept2_info = {'id':'789012', 'name':'Mechanical Engineering'}
department2 = Row(**dept2_info)
department2

Row(id='789012', name='Mechanical Engineering')

## Unpacking a list of row dictionaries

In [5]:
dept_info = [{'id':123456, 'name':'Computer Science'},
             {'id':789012, 'name':'Mechanical Engineering'},
             {'id':345678, 'name':'Theater and Drama'},
             {'id':901234, 'name':'Indoor Recreation'}]

dept_rows = [Row(**r) for r in dept_info]
dept_rows

[Row(id=123456, name='Computer Science'),
 Row(id=789012, name='Mechanical Engineering'),
 Row(id=345678, name='Theater and Drama'),
 Row(id=901234, name='Indoor Recreation')]

## Access `Row` content with column attributes

In [6]:
[dept.id for dept in dept_rows]

[123456, 789012, 345678, 901234]

In [7]:
[dept.name for dept in dept_rows]

['Computer Science',
 'Mechanical Engineering',
 'Theater and Drama',
 'Indoor Recreation']

## Creating a `pyspark.DataFrame`

* A `DataFrame` is a collection of `Row`s
* Create with spark.createDataFrame
* Need to have a 

In [8]:
df = spark.createDataFrame(dept_rows)
df

DataFrame[id: bigint, name: string]

## How to think about a `pyspark.DataFrame`

<img src="./img/pyspark_df.png" width=600>

## Example - `filter` and `collect`

In [34]:
output = (df
            .filter(df.name.startswith('C'))
            .collect())
output

[Row(id=123456, name='Computer Science')]

## Why is `pyspark` so slow

* Optimized for 
    * Distributed computation
    * Big data 
* Not great for
    * Local work
    * Small data

## `filter` and `collect` illustrated

<img src="./img/pyspark_filter_collect.gif" width=600>

## Reading a `csv` file with `spark.read.csv`

In [10]:
heros = spark.read.csv('./data/heroes_information.csv', header=True)
heros

DataFrame[Id: string, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: string, Publisher: string, Skin color: string, Alignment: string, Weight: string]

## Inspecting the column types

In [35]:
heros.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Eye color: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Hair color: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Skin color: string (nullable = true)
 |-- Alignment: string (nullable = true)
 |-- Weight: double (nullable = true)



## Gathering results in `pyspark.sql`

* **Important fact** All `pyspark` queries end in a collection method
* **Why?** Data is (possibly) spread across many machines
* <font color = "red"> **Warning** This might be is *expensive*! Know how much data your are requesting! </font>

## Gathering methods

* `collect` returns all values
* `take(n)` returns the first `n` values 
* `sample(n)` returns `n` randomly selected values 

## Inspecting the content - `take`

In [36]:
heros.take(5)

[Row(Id=0, name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='good', Weight=441.0),
 Row(Id=1, name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height=191.0, Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight=65.0),
 Row(Id=2, name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height=185.0, Publisher='DC Comics', Skin color='red', Alignment='good', Weight=90.0),
 Row(Id=3, name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=441.0),
 Row(Id=4, name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height=-99.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=-99.0)]

## Inspecting the content - `sample`

In [14]:
heros.sample(fraction=0.01).collect()

[Row(Id='21', name='Ando Masahashi', Gender='Male', Eye color='-', Race='-', Hair color='-', Height='-99.0', Publisher='NBC - Heroes', Skin color='-', Alignment='good', Weight='-99.0'),
 Row(Id='187', name='Cottonmouth', Gender='Male', Eye color='brown', Race='Human', Hair color='Black', Height='183.0', Publisher='Marvel Comics', Skin color='-', Alignment='bad', Weight='99.0'),
 Row(Id='222', name='Doctor Doom II', Gender='Male', Eye color='brown', Race='-', Hair color='Brown', Height='201.0', Publisher='Marvel Comics', Skin color='-', Alignment='bad', Weight='132.0'),
 Row(Id='574', name='Sandman', Gender='Male', Eye color='brown', Race='Human', Hair color='Brown', Height='185.0', Publisher='Marvel Comics', Skin color='-', Alignment='neutral', Weight='203.0'),
 Row(Id='600', name='Silver Surfer', Gender='Male', Eye color='white', Race='Alien', Hair color='No Hair', Height='193.0', Publisher='Marvel Comics', Skin color='silver', Alignment='good', Weight='101.0')]

## Getting all results with `collect`

<font color = "red"> **Warning** This might be is *expensive*! Know how much data your are requesting! </font> 

**The `collect` rule:** `count` before `collect`

In [ ]:
heros.collect() # <-- probably don't do this

In [ ]:
heros.filter(heros['Eye Color'] == 'blue').collect() # <-- better but still might be lots, let's check

In [40]:
heros.filter(heros['Eye Color'] == 'blue').count()

225

## Did you notice?

<img src="./img/pyspark_missing_values.png" width=400>

## Specifying a `nullValue`

In [15]:
heros = spark.read.csv('./data/heroes_information.csv', header=True, nullValue='-')
heros.take(5)

[Row(Id='0', name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height='203.0', Publisher='Marvel Comics', Skin color=None, Alignment='good', Weight='441.0'),
 Row(Id='1', name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height='191.0', Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight='65.0'),
 Row(Id='2', name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height='185.0', Publisher='DC Comics', Skin color='red', Alignment='good', Weight='90.0'),
 Row(Id='3', name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height='203.0', Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight='441.0'),
 Row(Id='4', name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height='-99.0', Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight='-99.0')

## Did you notice?

<img src="./img/pyspark_default_types.png" width=400>

Default type is a string

## Letting `spark` guess the types

Set `inferScheme=True` 

In [16]:
heros = spark.read.csv('./data/heroes_information.csv', header=True, inferSchema=True, nullValue='-')
heros

DataFrame[Id: int, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: double, Publisher: string, Skin color: string, Alignment: string, Weight: double]

## Checking the column types after `inferScheme`

In this case, `spark` guessed correctly

In [17]:
heros.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Eye color: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Hair color: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Skin color: string (nullable = true)
 |-- Alignment: string (nullable = true)
 |-- Weight: double (nullable = true)



## Inspecting the content - `take`

In [18]:
heros.take(5)

[Row(Id=0, name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='good', Weight=441.0),
 Row(Id=1, name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height=191.0, Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight=65.0),
 Row(Id=2, name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height=185.0, Publisher='DC Comics', Skin color='red', Alignment='good', Weight=90.0),
 Row(Id=3, name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=441.0),
 Row(Id=4, name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height=-99.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=-99.0)]

## Explicit `schema` specification

Format is `add(name, type, nullable?)`

In [41]:
from pyspark.sql.types import StructType
from pyspark.sql.types import DoubleType, StringType, IntegerType

hero_schema = (StructType()
  .add('Id', IntegerType(), False)
  .add('name', StringType(), True)
  .add('Gender', StringType(), True)
  .add('Eye color', StringType(), True)
  .add('Race', StringType(), True)
  .add('Hair color', StringType(), True)
  .add('Height', DoubleType(), True)
  .add('Publisher', StringType(), True)
  .add('Skin color', StringType(), True)
  .add('Alignment', StringType(), True)
  .add('Weight', DoubleType(), True))

heros = spark.read.csv('./data/heroes_information.csv', header=True, schema=hero_schema, nullValue='-')
heros

DataFrame[Id: int, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: double, Publisher: string, Skin color: string, Alignment: string, Weight: double]

In [42]:
heros.take(5)

[Row(Id=0, name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='good', Weight=441.0),
 Row(Id=1, name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height=191.0, Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight=65.0),
 Row(Id=2, name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height=185.0, Publisher='DC Comics', Skin color='red', Alignment='good', Weight=90.0),
 Row(Id=3, name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=441.0),
 Row(Id=4, name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height=-99.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=-99.0)]

## <font color="red"> Exercise 1 </font>

Define a `schema` and read in `./data/super_hero_powers.csv`

In [27]:
from pyspark.sql.types import StructType
from pyspark.sql.types import BooleanType, StringType 

# Your code here

## `pyspark.sql` queries are like `sqlalchemy` queries

#### Filter, group, and aggregate (categorical)

In [53]:
(heros
     .where(heros.Gender == 'Male')
     .groupby(heros['Eye color'])
     .count()
     .take(5))

[Row(Eye color='grey', count=6),
 Row(Eye color='green', count=30),
 Row(Eye color='yellow', count=16),
 Row(Eye color='bown', count=1),
 Row(Eye color=None, count=121)]

#### Group by multiple and aggregate (categorical)

In [54]:
(heros
     .groupby(heros['Eye color'], heros.Gender)
     .count()
     .take(5))

[Row(Eye color='yellow (without irises)', Gender=None, count=1),
 Row(Eye color='green', Gender='Male', count=30),
 Row(Eye color='violet', Gender='Female', count=2),
 Row(Eye color='hazel', Gender='Female', count=3),
 Row(Eye color='blue', Gender='Male', count=143)]

## <font color="red"> Exercise 2 </font>
    
Perform `pyspark.sql` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [43]:
# Your code here

# Appendix

## Creating rows from list of data

## Creating a Row class

* Pass `Row` the columns names
* Creates a specialized `Row` class

In [65]:
Employee = Row("firstName", "lastName", "email", "salary")
Employee

<Row(firstName, lastName, email, salary)>

## Creating a `Employee` instance

* Pass the data to `Employee` to make a row
* Order matters ... use the same order as names

In [22]:
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee1

Row(firstName='michael', lastName='armbrust', email='no-reply@berkeley.edu', salary=100000)

## Unpacking a data list

* Suppose the data is in a list/tuple.
* Use sequence unpacking with `*`

In [23]:
empl2_info = ('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
empl2_info

('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)

In [24]:
employee2 = Employee(*empl2_info)
employee2

Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000)

## Unpacking 

In [25]:
# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employees = [('michael', 'armbrust', 'no-reply@berkeley.edu', 100000),
             ('xiangrui', 'meng', 'no-reply@stanford.edu', 120000),
             ('matei', None, 'no-reply@waterloo.edu', 140000),
             (None, 'wendell', 'no-reply@berkeley.edu', 160000)]
emp_rows = [Employee(*r) for r in employees]
emp_rows

[Row(firstName='michael', lastName='armbrust', email='no-reply@berkeley.edu', salary=100000),
 Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000),
 Row(firstName='matei', lastName=None, email='no-reply@waterloo.edu', salary=140000),
 Row(firstName=None, lastName='wendell', email='no-reply@berkeley.edu', salary=160000)]